In [1]:
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None

In [2]:
test = pd.read_csv('test.csv') # load data
train = pd.read_csv('train.csv') # load data

In [3]:
df = pd.concat([train,test], sort = False) # merge test and train into one df

In [4]:
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True) # convert string date into datetime
df['month'] = df['date'].dt.month # get month 
df['weekday'] = df['date'].dt.dayofweek # get day of week
df['year'] = df['date'].dt.year # get year
df['week_of_year']  = df.date.dt.weekofyear # get week number

In [5]:
df.set_index("date",inplace=True) # make date index

In [6]:
df["median-store_item-month"] = df.groupby(['month',"item","store"])["sales"].transform("median")
df["mean-store_item-week"] = df.groupby(['week_of_year',"item","store"])["sales"].transform("mean")
df["item-month-sum"] = df.groupby(['month',"item"])["sales"].transform("sum") # total sales of that item  for all stores
df["store-month-sum"] = df.groupby(['month',"store"])["sales"].transform("sum") # total sales of that store  for all items

In [7]:
# get shifted features for grouped data. Note need to sort first! 
df['store_item_shifted-90'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90)) # sales for that item 90 days = 3 months ago
df['store_item_shifted-180'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(180)) # sales for that item 180 days = 6 months ago
df['store_item_shifted-365'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(365)) # sales for that 1 year  ago

df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).sum()) # shifted total sales for that item 12 weeks (3 months) ago
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).sum()) # shifted total sales for that store 12 weeks (3 months) ago
df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).mean()) # shifted mean sales for that item 12 weeks (3 months) ago
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).mean()) # shifted mean sales for that store 12 weeks (3 months) ago

In [8]:
col = [i for i in df.columns if i not in ['date', 'id']] # get columns that we need 
y = 'sales'

In [9]:
train = df.loc[~df.sales.isna()] # if sales is not null we get train set with sales
test = df.loc[df.sales.isna()] # if sales is null we get test set without sales 
test.drop('sales', axis=1, inplace=True) # get rid of sales
train.drop('id', axis=1, inplace=True) # get rid of id
train['store_item_shifted-90'] = train['store_item_shifted-90'].fillna(train['store_item_shifted-90'].mean()) # get rid of NaN values
train['store_item_shifted-180'] = train['store_item_shifted-180'].fillna(train['store_item_shifted-180'].mean()) # get rid of NaN values
train['store_item_shifted-365'] = train['store_item_shifted-365'].fillna(train['store_item_shifted-365'].mean()) # get rid of NaN values

In [10]:
x_train, x_test, y_train, y_test = train_test_split(train[col], train[y], test_size=0.2) # split into train and test

In [11]:
reg = LinearRegression() # get model
reg.fit(x_train, y_train) # train model with splitted data
reg.score(x_test, y_test) # get accuracy of model

1.0

In [12]:
test.reset_index(drop=True, inplace=True) # get rid of index
ind = test.id # get id
test.drop('id', axis=1, inplace=True) # get rid of id

In [13]:
reg.fit(train.drop('sales', axis=1), train.sales) #train pur model with full train set

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [14]:
predict = reg.predict(test) # predict values
preid = list(zip(ind, predict)) # merge id and values

In [15]:
subm = pd.DataFrame(preid, columns = ['id', 'sales']) #convert list of tuples to df
subm.id = subm.id.astype(int) # convert float column into int
subm.sales = subm.sales.round().astype(int) # round all values in column and convert to int
subm.to_csv('submission.csv') # convert df to csv